<a href="https://colab.research.google.com/github/deiteris/voice-changer/blob/master-custom/Colab_RealtimeVoiceChanger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### [w-okada's Voice Changer](https://github.com/deiteris/voice-changer) | **Google Colab**

---

## **READ ME - VERY IMPORTANT**

You can use the following settings for optimal results:

Best performance: `f0: fcpe | Chunk: 64.0ms or higher | Extra: 2.7s`<br>
Best quality: `f0: rmvpe | Chunk: 72.0ms or higher | Extra: 5s`<br>
**Don't forget to select your Colab GPU in the GPU field (<b>Tesla T4</b>, for free users)*

You can tune `Chunk` for lower/higher delay and `Extra` for better quality.

*You can always [click here](https://rentry.co/VoiceChangerGuide#gpu-chart-for-known-working-chunkextra) to check if these settings are up-to-date*
<br><br>

---

### Always use Colab GPU (**VERY VERY VERY IMPORTANT!**)
You need to use a Colab GPU so the Voice Changer can work faster and better\
Use the menu above and click on **Runtime** » **Change runtime** » **Hardware acceleration** to select a GPU (**T4 is the free one**)

---

# **Credits and Support**
Realtime Voice Changer by [w-okada](https://github.com/w-okada)<br>
Original instructions by [Hina](https://github.com/HinaBl)<br>

Need help? [AI Hub Discord](https://discord.gg/aihub) » ***#help-realtime-vc***

---

In [ ]:
#=================Updated=================
# @title **[1]** Clone repository and install dependencies
# @markdown This first step will download the latest version of Voice Changer and install the dependencies. **It can take some time to complete.**
import os
import time
import threading
from google.colab import drive

# Configs
Run_Cell=0

#@markdown ---
# @title **[Optional]** Connect to Google Drive
# @markdown Using Google Drive will automatically save your uploaded models for later use. Make sure you have sufficient amount of space on your Google Drive.

Use_Drive=True #@param {type:"boolean"}

from IPython.display import clear_output, Javascript

def update_timer_and_print():
    global timer
    while True:
        hours, remainder = divmod(timer, 3600)
        minutes, seconds = divmod(remainder, 60)
        timer_str = f'{hours:02}:{minutes:02}:{seconds:02}'
        print(f'\rTimer: {timer_str}', end='', flush=True)  # Print without a newline
        time.sleep(1)
        timer += 1
timer = 0
threading.Thread(target=update_timer_and_print, daemon=True).start()

!pip install colorama --quiet
from colorama import Fore, Style

print(f"{Fore.CYAN}> Downloading prebuilt executable...{Style.RESET_ALL}")

import requests

res = requests.get('https://api.github.com/repos/deiteris/voice-changer/releases/latest')
release_info = res.json()

for asset in release_info['assets']:
   if not asset['name'].startswith('voice-changer-linux-amd64-cuda.tar.gz'):
      continue
   download_url = asset['browser_download_url']
   !wget -q --show-progress {download_url}

print(f"{Fore.GREEN}> Unpacking...{Style.RESET_ALL}")
!cat voice-changer-linux-amd64-cuda.tar.gz.* | tar xzf -
print(f"{Fore.GREEN}> Finished unpacking!{Style.RESET_ALL}")
!rm -rf voice-changer-linux-amd64-cuda.tar.gz.*

%cd MMVCServerSIO

if Use_Drive and not os.path.exists('/content/drive'):
  drive.mount('/content/drive')

  !mkdir -p /content/drive/MyDrive/voice-changer/server/model_dir
  !mkdir -p /content/drive/MyDrive/voice-changer/server/pretrain
  !rm -rf /content/voice-changer/server/model_dir
  !rm -rf /content/voice-changer/server/pretrain

  time.sleep(5)

  os.symlink("/content/drive/MyDrive/voice-changer/server/model_dir", "/content/voice-changer/server/model_dir", True)
  os.symlink("/content/drive/MyDrive/voice-changer/server/pretrain", "/content/voice-changer/server/pretrain", True)

print(f"{Fore.GREEN}> Successfully downloaded and unpacked the binary!{Style.RESET_ALL}")

print(f"{Fore.CYAN}> Installing libportaudio2...{Style.RESET_ALL}")
!apt-get -y install libportaudio2 -qq
print(f"{Fore.GREEN}> Successfully installed all packages!{Style.RESET_ALL}")

In [ ]:
#=================Updated=================
# @title **[2]** Set server configuration
# @markdown This cell will set the server configuration.

print(f"{Fore.CYAN}> Installing pre-dependencies...{Style.RESET_ALL}")
!pip install python-dotenv --quiet

%cd /content/MMVCServerSIO

from dotenv import set_key

set_key('.env', "ALLOWED_ORIGINS", "*")
set_key('.env', "SAMPLE_MODE", "")

Ready = True

In [ ]:
print(f"{Fore.CYAN}> Installing pre-dependencies...{Style.RESET_ALL}")
!pip install pyngrok --quiet
#=======================Updated=========================

# @title Start Server **using ngrok**
# @markdown This cell will start the server, the first time that you run it will download the models, so it can take a few minutes (usually ~1-2 minutes)

# @markdown ---
# @markdown You'll need a ngrok account, but <font color=green>**it's free**</font> and easy to create!
# @markdown ---
# @markdown **1** - Create a <font color=green>**free**</font> account at [ngrok](https://dashboard.ngrok.com/signup) or **login with Google/Github account**\
# @markdown **2** - If you didn't logged in with Google/Github, you will need to **verify your e-mail**!\
# @markdown **3** - Click [this link](https://dashboard.ngrok.com/get-started/your-authtoken) to get your auth token, and place it here:
Token = 'TOKEN_HERE' # @param {type:"string"}
# @markdown **4** - *(optional)* Change to a region near to you\
# @markdown `Default Region: ap - Asia/Pacific (Singapore)`
Region = "ap - Asia/Pacific (Singapore)" # @param ["ap - Asia/Pacific (Singapore)", "au - Australia (Sydney)","eu - Europe (Frankfurt)", "in - India (Mumbai)","jp - Japan (Tokyo)","sa - South America (Sao Paulo)", "us - United States (Ohio)"]

#@markdown **5** - *(optional)* Other options:
ClearConsole = True  # @param {type:"boolean"}
Play_Notification = False  # @param {type:"boolean"}

# ---------------------------------
# DO NOT TOUCH ANYTHING DOWN BELOW!
# ---------------------------------

# Check if Run_Cell
if 'Ready' not in globals() or not Ready:
    print("Go back and run first and second cells.")
else:
    from pyngrok import conf, ngrok
    MyConfig = conf.PyngrokConfig()
    MyConfig.auth_token = Token
    MyConfig.region = Region[0:2]
    conf.get_default().authtoken = Token
    conf.get_default().region = Region
    conf.set_default(MyConfig);

    import subprocess, threading, time, socket, urllib.request
    PORT = 18888

    from pyngrok import ngrok
    ngrokConnection = ngrok.connect(PORT)
    public_url = ngrokConnection.public_url

    from IPython.display import clear_output
    from IPython.display import Audio, display
    def play_notification_sound():
        display(Audio(url='https://raw.githubusercontent.com/hinabl/rmvpe-ai-kaggle/main/custom/audios/notif.mp3', autoplay=True))


    def wait_for_server():
        while True:
            time.sleep(0.5)
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            result = sock.connect_ex(('127.0.0.1', PORT))
            if result == 0:
                break
            sock.close()
        if ClearConsole:
            clear_output()
        print("--------- SERVER READY! ---------")
        print("Your server is available at:")
        print(public_url)
        print("---------------------------------")
        if Play_Notification==True:
          play_notification_sound()

    threading.Thread(target=wait_for_server, daemon=True).start()

    !./MMVCServerSIO

    ngrok.disconnect(ngrokConnection.public_url)